In [1]:
!pip install tensorflow==2.15 --quiet
!pip install keras==2.15 --quiet
!pip install tf-keras==2.15 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.68 requires ml-dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.2 MB/s eta 0:00:00


In [2]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

In [3]:
drive.mount('/content/drive')

data_dir = "/content/drive/My Drive/Capstone_kitaa/ingredients_recipe/"

Mounted at /content/drive


In [4]:
# Augmentasi data untuk memperkaya dataset
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Membuat generator data untuk training
train_generator = datagen.flow_from_directory(
    directory=data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Membuat generator data untuk validasi (gunakan data yang terpisah untuk validasi)
validation_generator = datagen.flow_from_directory(
    directory='/content/drive/My Drive/Capstone_kitaa/ingredients_recipe/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 2639 images belonging to 53 classes.
Found 2639 images belonging to 53 classes.


In [5]:
# Feature Extractor menggunakan MobileNetV2
def feature_extractor(inputs):
    mobilenet_model = tf.keras.applications.MobileNetV2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    mobilenet_model.trainable = False
    feature_extractor = mobilenet_model(inputs)
    return feature_extractor

# Dense layers untuk pengklasifikasian
def dense_layers(features):
    x = tf.keras.layers.GlobalAveragePooling2D()(features)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    return x

# Output bounding box atau klasifikasi
def final_model(inputs):
    feature_cnn = feature_extractor(inputs)
    x = dense_layers(feature_cnn)
    outputs = tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Compile model
inputs = tf.keras.Input(shape=(224, 224, 3))
model = final_model(inputs)

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                

In [7]:
history = model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
82/82 [==============================] - 883s 11s/step - loss: 2.0659 - accuracy: 0.4772 - val_loss: 1.0001 - val_accuracy: 0.7001
Epoch 2/10
82/82 [==============================] - 132s 2s/step - loss: 0.9813 - accuracy: 0.6997 - val_loss: 0.6948 - val_accuracy: 0.7950
Epoch 3/10
82/82 [==============================] - 100s 1s/step - loss: 0.6853 - accuracy: 0.7967 - val_loss: 0.5360 - val_accuracy: 0.8342
Epoch 4/10
82/82 [==============================] - 101s 1s/step - loss: 0.5096 - accuracy: 0.8412 - val_loss: 0.4063 - val_accuracy: 0.8700
Epoch 5/10
82/82 [==============================] - 100s 1s/step - loss: 0.4527 - accuracy: 0.8535 - val_loss: 0.3433 - val_accuracy: 0.8880
Epoch 6/10
82/82 [==============================] - 100s 1s/step - loss: 0.3800 - accuracy: 0.8776 - val_loss: 0.3136 - val_accuracy: 0.8979
Epoch 7/10
82/82 [==============================] - 132s 2s/step - loss: 0.3130 - accuracy: 0.9007 - val_loss: 0.3345 - val_accuracy: 0.9040
Epoch 8/10
8

In [12]:
# Simpan model yang sudah dilatih
model.save('dapur_pintar_model.h5')

model.save('/content/drive/My Drive/Capstone_kitaa/ingredients_recipe/dapur_pintar_model.h5')

In [13]:
from google.colab import files
files.download('dapur_pintar_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>